In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(2)

dataset_size = ["small", "medium", "large"][0]

dataset_info = {
    "small": {
        "dataset_name": "wine",
        "class_name": "Class",
        "drop_fields": []
    },
    "medium": {
        "dataset_name": "breast-cancer-wisconsin",
        "class_name": "Class",
        "drop_fields": ["Sample code number"]
    },
    "large": {
        "dataset_name": "seismic-bumps",
        "class_name": "class",
        "drop_fields": []
    }
}

dataset_name = dataset_info[dataset_size]["dataset_name"]
class_name = dataset_info[dataset_size]["class_name"]
drop_fields = dataset_info[dataset_size]["drop_fields"]

df = pd.read_csv('../data/' + dataset_name + ".csv")
df = df.drop(drop_fields, axis=1)
df = df.iloc[np.random.permutation(len(df))]

n_cut = int(0.8*len(df))
df_trn = df[:n_cut]
df_tst = df[n_cut:]

X_trn = df_trn.drop(class_name, axis=1)
y_trn = df_trn[class_name]

X_tst = df_tst.drop(class_name, axis=1)
y_tst = df_tst[class_name]

In [4]:
X = [['blue', 'blonde', 'tall'],
     ['blue', 'brown', 'medium'],
     ['brown', 'brown', 'medium'],
     ['green', 'brown', 'medium'],
     ['green', 'brown', 'tall'],
     ['brown', 'brown', 'low'], 
     ['green', 'blonde', 'low'],
     ['blue', 'brown', 'medium']]

X = np.array(X)
y = np.array([1, 1, 0, 0, 1, 0, 0, 1])

In [1]:
def Gini_X(X, y):
    G = 1
    for cls in np.unique(y):
        G -= ((y == cls).sum() / len(X))**2
    return G


def Gini_X_A(X, y, A):
    features, values = A
    feature1, feature2 = features
    values1, values2 = values
    
    ind1 = np.isin(X[:, feature1], values1) 
    X1 = X[ind1, :]
    y1 = y[ind1]
    
    ind2 = np.isin(X[:, feature2], values2) 
    X2 = X[ind2, :]
    y2 = y[ind2]
    
    return len(X1) / len(X) * Gini_X(X1, y1) + len(X2) / len(X) * Gini_X(X2, y2)


In [10]:
Gini_X(X, y)

0.5